## Import dependencies

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy  # policy based rf
from rl.memory import SequentialMemory
from racing_car_env import RacingCarEnv
import numpy as np
import os

2023-06-05 01:29:07.258134: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-05 01:29:07.322593: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-05 01:29:07.324142: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 01:29:08.335858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


pygame 2.3.0 (SDL 2.24.2, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = RacingCarEnv()
states = env.observation_space.shape[0]
actions = env.action_space.n

/home/kuba/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


## Build model and agent

In [3]:
def build_model(states, actions, depth_sizes = (64, 64)):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    for size in depth_sizes:
        model.add(Dense(size, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [4]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)

    return dqn

In [5]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 5)                 0         
                                                                 
 dense (Dense)               (None, 64)                384       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 4,739
Trainable params: 4,739
Non-trainable params: 0
_________________________________________________________________


In [6]:
dqn = build_agent(model, actions)

In [7]:
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

2023-06-05 01:29:18.297135: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-05 01:29:18.339154: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_2_1/kernel/Assign' id:139 op device:{requested: '', assigned: ''} def:{{{node dense_2_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2_1/kernel, dense_2_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Train model

In [14]:
dqn.fit(env, nb_steps=30000, visualize=False, verbose=1)

Training for 30000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 10.1784
150 episodes - episode_reward: 675.166 [182.860, 2681.880] - loss: 11604.635 - mae: 1294.423 - mean_q: 1970.678

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 10.1809
138 episodes - episode_reward: 737.227 [223.564, 3039.591] - loss: 35860.402 - mae: 2511.031 - mean_q: 3828.342

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 10.1808
done, took 178.958 seconds


## Save and load weights

In [15]:
custom_model_path = os.path.join('Training', 'Saved_models', 'car_model_custom')

In [16]:
dqn.save_weights(custom_model_path, True)

In [17]:
dqn.load_weights(custom_model_path)

## Test model

In [18]:
scores = dqn.test(env, nb_episodes=2, visualize=True)
print(np.mean(scores.history['episode_reward']))
env.close()

Testing for 2 episodes ...
Episode 1: reward: 6209.437, steps: 610
Episode 2: reward: 6209.437, steps: 610
6209.43699999999
